# 卷积
对于图片来说，不适合使用MLP来处理，因为参数过多导致难以拟合，卷积神经网络Convolutional Neural Networks，是机器学习中利用一些已知结构的创造性方法。

卷积神经网络正式将空间不变性这一概念系统化，基于这个模型使用较少的参数来学习有用的表示：

inductive bias

1. 平移不变性：图像经过平移，相应的特征图上的表达也是平移的
2. 局部性：神经网络的前面几层应该只探索输入图像的局部区域，而不是过度在意图像中相隔较远区域的关系，最终可以聚合这些特征，以在整个图像级别进行预测


# 平移不变性
如果一幅图片是h*w*1的，姑且算是单通道，那么用MLP的话，要输出h'*w'*1的数据，我们需要h*w*h'*w'的数据，这将是爆炸的。
如果利用平移不变性的话，至少可以将参数缩减到h*w

# 局部性
我们不应该偏离到很远的地方，那么还可以进一步缩小参数,可以将h*w缩小很多
$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$$

# 卷积
在数学中，两个函数的卷积被定义为
$$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.$$

卷积就是把一个函数“翻转”并位移X时，测量f和g之间的重叠，见
https://www.zhihu.com/question/22298352

In [21]:
# 实现coor2d函数，接受张量X和卷积核张量K，并返回张量Y
import torch as t
import torch.nn as nn
from torch import Tensor


def corr2d(X: Tensor, K: Tensor) -> Tensor:
    """计算二维互相关运算,len(X.shape)=len(K.shape)=2"""
    h,w=K.shape
    Y = t.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y


In [22]:
# 尝试计算一下？
X = t.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K= t.tensor([[0.,1.],[2.,3.]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [23]:
# 定义卷积层
class CONV2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = nn.Parameter(t.rand(kernel_size))
        self.bias = nn.Parameter(t.zeros(1))

    def forward(self,x:Tensor)->Tensor:
        return corr2d(x,self.weight)+self.bias

In [24]:
X = t.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [25]:
K=t.tensor([[1.,-1.]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [26]:
# 转置之后，失去了其相应的判别能力
corr2d(X.t(), K)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [27]:
# 学习卷积核的参数
conv2d =nn.Conv2d(1,1,(1,2),bias=False)

X=X.reshape((1,1,6,8))
Y= Y.reshape((1,1,6,7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=lr*conv2d.weight.grad
    print(f'epoch {i+1}, loss {l.sum():.3f}')


epoch 1, loss 29.116
epoch 2, loss 13.258
epoch 3, loss 6.283
epoch 4, loss 3.119
epoch 5, loss 1.627
epoch 6, loss 0.890
epoch 7, loss 0.508
epoch 8, loss 0.299
epoch 9, loss 0.181
epoch 10, loss 0.112


In [28]:
for param in conv2d.parameters():
    print(param)

Parameter containing:
tensor([[[[ 0.9506, -1.0173]]]], requires_grad=True)


# 互相关和卷积
他俩是不同的，卷积运算需要水平和垂直翻转卷积核张量，然后对输入张量执行互相关运算

# 特征映射和感受野
输出的卷积层有时候被称为“特征映射” feature map，对于某一层的任意元素X，其感受野receptive field 是指在前向传播期间可能影响X计算的所有元素。

In [29]:
# 练习
X = t.eye(5)
corr2d(X,K)


tensor([[ 1.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.],
        [ 0., -1.,  1.,  0.],
        [ 0.,  0., -1.,  1.],
        [ 0.,  0.,  0., -1.]])

In [30]:
corr2d(X,K.T)

tensor([[ 1., -1.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  1., -1.]])